In [1]:
import csv
import pymongo
from pymongo import MongoClient
import datetime as dt
import calendar
import pandas as pd
import numpy as np
import time
#These are all the imports to be used

In [2]:
c_v1 = MongoClient("mongodb://3.20.189.28:2036/momoKashBd?readPreference=secondary&directConnection=true")
db_v1 = c_v1["momoKashBd"]
#These are my collections
#Clients = db_v1["Client"]
#Loans = db_v1["Loan"]
Refunds = db_v1["Refund"]
print("MongoDB connection established with success")

MongoDB connection established with success


In [3]:
#here we define the date to be used 
date_start = dt.datetime(2023, 8, 1)
date_end = dt.datetime(2023, 8, 30)

In [4]:
# Inspect one document to see field names and types
Refunds.find_one()

{'_id': '237650000024:CAM07X02T20221224022010-RM:5289810799',
 'momoTransactionID': '5289810799',
 'msisdn': '237650000024',
 'clientName': 'MAGNUS YONGHABI YUH',
 'bankID': '3',
 'bankName': 'CAMCCUL',
 'amount': 4.0,
 'date': '24/12/2022',
 'refundingID': 'CAM07X02T20221224022010-RM',
 'loanKey': '237650000024:CAM07X01T20220422043548MU:3933818854',
 'bonusAmount': 3150.0,
 'refundPeriod': 'Carried Over',
 'balance': 3463.0,
 'excess': 0.0,
 'carryoverDate': '22/05/2022',
 'loanDate': '22/04/2022',
 'refundType': 'USSD_P',
 'refundCapital': 3.0,
 'loanPolicyID': '1',
 'refundInterest': 1.0,
 'systemUsed': 'USSD',
 'timestamp': '1671848410',
 'reconductionDate': '07/05/2022',
 'processingNumber': '20221224022010RM',
 'time': '02:20:10',
 'clientID': '650000024',
 'loanID': 'CAM07X01T20220422043548MU',
 'disbursementId': '2022121645405473M50000024',
 'status': 'COMPLETED',
 'language': '1',
 'bankCode': 'CAM07X01T',
 'dateTime': datetime.datetime(2022, 12, 24, 2, 20, 10)}

In [5]:
# Cell 5 - Fetching raw refunds from Mongo with Python-side date filter (using dateTime + status)

start = time.time()

# Fetch refunds
cursor = Refunds.find(
    { 
        "dateTime": {"$gte": date_start, "$lte": date_end},  # filtering by dateTime
        "status": "COMPLETED"  #only completed refunds
    },
    {
        "_id": 0,
        "clientID": 1,
        "loanID": 1,
        "amount": 1,
        "refundCapital": 1,
        "refundInterest": 1,
        "balance": 1,
        "refundPeriod": 1,
        "refundType": 1,
        "loanDate": 1,
        "date": 1,
        "status": 1,
        "excess": 1,
        "dateTime": 1
    }
)

refunds_df = pd.DataFrame(list(cursor))

#Check if empty
if refunds_df.empty:
    print("No completed refunds were made during this period.")
else:
    # Convert string date fields → datetime
    for col in ["loanDate", "date"]:
        refunds_df[col] = pd.to_datetime(refunds_df[col], dayfirst=True, errors="coerce")

    # Sort refunds by dateTime
    refunds_df = refunds_df.sort_values("dateTime", ascending=True)

    print("Fetched completed refunds:", refunds_df.shape)
    print(refunds_df.head(3))  # show a small sample

end = time.time()
print(f"Fetching time: {round((end-start)/60, 2)} minutes")


Fetched completed refunds: (405832, 13)
    clientID                                             loanID   amount  \
0  671355813  237671355813:CAM07X01T202305221418070617MP7135...   4594.0   
1  680539379  237680539379:CAM07X01T202307250033420893MP8053...  16125.0   
2  679432461  237679432461:CAM07X01T202307301027070979MP7943...   3000.0   

   balance  excess       date refundPeriod     refundType   loanDate  \
0      0.0     0.0 2023-08-01  Reconducted  AUTODEBIT_ECW 2023-05-22   
1      0.0     0.0 2023-08-01     Promo On           USSD 2023-07-25   
2   5000.0     0.0 2023-08-01     Promo On           USSD 2023-07-30   

   refundInterest  refundCapital     status            dateTime  
0          320.51        4273.49  COMPLETED 2023-08-01 00:00:09  
1         1125.00       15000.00  COMPLETED 2023-08-01 00:01:17  
2            0.00        3000.00  COMPLETED 2023-08-01 00:01:23  
Fetching time: 18.1 minutes


In [6]:
# Cell 6 - Clean + filter refunds

# Columns with string dates
date_cols = ["loanDate", "date"]
for col in date_cols:
    refunds_df[col] = pd.to_datetime(refunds_df[col], dayfirst=True, errors="coerce")

# Ensure new fields exist (fill with 0 if missing)
req_fields = ["excess", "balance"]
for col in req_fields:
    if col not in refunds_df.columns:
        refunds_df[col] = 0
    else:
        refunds_df[col] = refunds_df[col].fillna(0)

# Fill missing numeric values in the standard fields
num_cols = ["amount", "refundCapital", "refundInterest", "balance", "excess"]
for col in num_cols:
    if col in refunds_df.columns:
        refunds_df[col] = refunds_df[col].fillna(0)

# Optional: filter by dateTime window again (safety check)
refunds_df = refunds_df[
    (refunds_df["dateTime"] >= date_start) &
    (refunds_df["dateTime"] <= date_end)
]

# Sort like the boss does
refunds_df = refunds_df.sort_values("dateTime", ascending=True)

print("Cleaned refunds in date window:", refunds_df.shape)
print(refunds_df.head(5))


Cleaned refunds in date window: (405832, 13)
    clientID                                             loanID   amount  \
0  671355813  237671355813:CAM07X01T202305221418070617MP7135...   4594.0   
1  680539379  237680539379:CAM07X01T202307250033420893MP8053...  16125.0   
2  679432461  237679432461:CAM07X01T202307301027070979MP7943...   3000.0   
3  672470791  237672470791:CAM07X01T202306241031510240MP7247...  11000.0   
4  653586212  237653586212:CAM07X01T202306300637160174MP5358...  10043.0   

   balance  excess       date refundPeriod     refundType   loanDate  \
0      0.0     0.0 2023-08-01  Reconducted  AUTODEBIT_ECW 2023-05-22   
1      0.0     0.0 2023-08-01     Promo On           USSD 2023-07-25   
2   5000.0     0.0 2023-08-01     Promo On           USSD 2023-07-30   
3    785.0     0.0 2023-08-01  isCarryOver  AUTODEBIT_ECW 2023-06-24   
4      0.0     0.0 2023-08-01     Promo On  AUTODEBIT_ECW 2023-06-30   

   refundInterest  refundCapital     status            dateTime  

In [7]:
# Cell 7 - Final cleaning + sample view for Refunds

# Convert date fields properly
refunds_df["loanDate"] = pd.to_datetime(refunds_df["loanDate"], errors="coerce")
refunds_df["date"] = pd.to_datetime(refunds_df["date"], errors="coerce")

# Ensure numeric fields are clean
numeric_cols = ["amount","refundCapital","refundInterest","balance","excess"]
for col in numeric_cols:
    refunds_df[col] = refunds_df[col].fillna(0).apply(lambda x: max(x, 0))

# Clean text fields (IDs, status, refundType, etc.)
text_cols = ["loanID","clientID","refundPeriod","refundType","status"]
for col in text_cols:
    if col in refunds_df.columns:
        refunds_df[col] = refunds_df[col].astype(str).str.strip()

# Reset index for a neat table
refunds_df.index = np.arange(refunds_df.shape[0])

# Optional: sort by refund date (chronological order)
refunds_df = refunds_df.sort_values("date", ascending=True)

# Show a random sample of fully cleaned refunds
print("Sample of fully cleaned refunds:")
display(refunds_df.sample(10))


Sample of fully cleaned refunds:


,clientID,loanID,amount,balance,excess,date,refundPeriod,refundType,loanDate,refundInterest,refundCapital,status,dateTime
141000,673316413,237673316413:CAM07X01T202307131419440462MP7331...,15188.0,0.0,0.0,2023-08-11,Reconducted,USSD,2023-07-13,1059.78,14128.22,COMPLETED,2023-08-11 06:06:02
48157,678269471,237678269471:NA05X01T202307050948400727MP78269...,15.0,9660.0,0.0,2023-08-04,isCarryOver,AUTODEBIT_ECW,2023-07-05,1.05,13.95,COMPLETED,2023-08-04 08:53:03
321961,674438604,237674438604:CAM07X01T202308230900250134MP7443...,6400.0,0.0,0.0,2023-08-26,Promo On,USSD,2023-08-23,0.00,6400.00,COMPLETED,2023-08-26 09:42:27
164192,,237678382273:CAM07X01T202308020114520685MP7838...,3000.0,24250.0,0.0,2023-08-12,Promo On,USSD,2023-08-02,209.30,2790.70,COMPLETED,2023-08-12 16:56:45
279886,678314703,237678314703:CAM07X01T202305201736460734MP7831...,423.0,0.0,0.0,2023-08-22,Reconducted,AUTODEBIT_ECW,2023-05-20,55.17,367.83,COMPLETED,2023-08-22 16:26:24
94700,672182247,237672182247:CAM07X01T202305241422040045MP7218...,2000.0,2755.0,0.0,2023-08-07,Reconducted,AUTODEBIT_ECW,2023-05-24,139.53,1860.47,COMPLETED,2023-08-07 07:27:56
186332,672825178,237672825178:CAM07X01T202306161347340929MP7282...,2.0,3213.0,0.0,2023-08-14,Reconducted,AUTODEBIT_ECW,2023-06-16,0.14,1.86,COMPLETED,2023-08-14 14:14:04
37235,672648500,237672648500:CAM07X01T202307121229310418MP7264...,6000.0,4750.0,0.0,2023-08-03,Promo On,USSD,2023-07-12,418.60,5581.40,COMPLETED,2023-08-03 16:12:33
393679,683917532,237683917532:RB011X01T20221025162046MU:4936322824,46.0,2015.0,0.0,2023-08-29,isCarryOver,AUTODEBIT_ECW,2022-10-25,3.21,42.79,COMPLETED,2023-08-29 21:51:49
245621,650988955,237650988955:CAM07X01T202305290637450243MP5098...,2500.0,1246.0,0.0,2023-08-19,Reconducted,AUTODEBIT_ECW,2023-05-29,174.42,2325.58,COMPLETED,2023-08-19 16:37:08


In [8]:
# Printing the payload csv
refunds_df.to_csv("refunds_aug_2023.csv", index=False, sep=",")
print("csv export aug 2023 complete")


csv export july 2023 complete
